In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [42]:
class TradingStrategy:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import os

    def __init__(self, data):
        self.data = data
        
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def trail_function(self, sell_rate, buy_rate):
        
        # Variables for tracking orders and positions
        orders = []
        current_position = "Buy"
        
        # Initial conditions
        interval_max_price = self.data.iloc[0]     # self.data[0]
        stop_price_sell = interval_max_price * (1 - sell_rate)
        stop_price_buy = interval_max_price * (1 + buy_rate)
        
        # Trading loop
        for day_price in self.data:
            if current_position == "Buy":
                if day_price > interval_max_price:
                    interval_max_price = day_price
                    stop_price_sell = interval_max_price * (1 - sell_rate)

                elif day_price <= stop_price_sell:
                    orders.append(("Sell", day_price))
                    current_position = "Sell"
                    interval_max_price = day_price
                    stop_price_buy = interval_max_price * (1 + buy_rate)

            elif current_position == "Sell":
                if day_price < interval_max_price:
                    interval_max_price = day_price
                    stop_price_buy = interval_max_price * (1 + buy_rate)

                elif day_price >= stop_price_buy:
                    orders.append(("Buy", day_price))
                    current_position = "Buy"
                    interval_max_price = day_price
                    stop_price_sell = interval_max_price * (1 - sell_rate)
        return orders
        
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def calculate_total_profit(self, trade_dict):
        total_profit = 0
        initial_price = None

        for action, price in trade_dict:
            if action == 'Buy':
                initial_price = price
            elif action == 'Sell' and initial_price is not None:
                total_profit += (price / initial_price) - 1
                initial_price = price
        return total_profit
        
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def calculate_profit(self, trade_dict):
        transaction_returns = []
        initial_price = None

        for action, price in trade_dict:
            if action == 'Buy':
                initial_price = price
            elif action == 'Sell' and initial_price is not None:
                transaction_returns.append((price / initial_price) - 1)
                initial_price = price
        return transaction_returns

    ####################### ####################### ####################### ####################### ####################### ####################### 
    
    def batch_from_end(self, lst, batch_size):
        # Calculate how many elements will be in the first (potentially smaller) batch
        first_batch_size = len(lst) % batch_size
        if first_batch_size == 0:
            first_batch_size = batch_size
        
        # Get the first batch
        result = [lst[:first_batch_size]]
        
        # Get the remaining batches
        for i in range(first_batch_size, len(lst), batch_size):
            result.append(lst[i:i + batch_size])
        
        return result
        
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    
    def batch_separator(self, list_of_batches):
        lines = []
        
        first_batch = list_of_batches[0]
        lines.append(first_batch.iloc[0])
        
        for batch in list_of_batches:
            lines.append(batch.iloc[-1])
            
        return lines
        
############################
    
    # def batch_separator(self, list_of_batches):
    #     lines = []
    #     if not list_of_batches:  # Check if list is empty
    #         return lines
        
    #     # If list_of_batches contains pandas Series
    #     first_batch = list_of_batches[0]
    #     if isinstance(first_batch, pd.Series):
    #         lines.append(first_batch.iloc[0])  # Use iloc for integer indexing with Series
    #         for batch in list_of_batches[1:]:
    #             lines.append(batch.iloc[-1])
    #     else:
    #         # If list_of_batches contains regular lists/arrays
    #         lines.append(first_batch[0])
    #         for batch in list_of_batches[1:]:
    #             lines.append(batch[-1])
        
    #     return lines
    
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def optimize_trading_rates(self, buy_rate_start=0.01, data_external=None):
        if data_external is not None and not data_external.empty:
            data = data_external
        else:
            data = self.data

        # Initialize variables
        returns = []
        sell_rate = []
        num_transactions = []
        Trading_Strategy = TradingStrategy(data)
        
        # Calculate returns and transactions for different sell rates
        for i in range(0, 100, 1):  # Using integers and dividing by 100
            i = i / 100.0
            sell_rate.append(i)
            returns.append(Trading_Strategy.calculate_total_profit(Trading_Strategy.trail_function(i, buy_rate_start)))
            num_transactions.append(len(Trading_Strategy.trail_function(i, buy_rate_start)))
    
        # Calculate transaction index for sell rates
        transaction_index = [
            (returns[j] / num_transactions[j]) if num_transactions[j] != 0 else 0 
            for j in range(len(sell_rate))
        ]
    
        # Find the optimized sell rate
        transaction_index_array = np.array(transaction_index)
        index = np.where(transaction_index_array == np.max(transaction_index_array))[0]
        optimized_sell_rate = sell_rate[index[0]]    


        
        # Initialize variables for buy rate optimization
        returns_Buy = []
        buy_rate = []
        num_transactions_Buy = []
    
        # Calculate returns and transactions for different buy rates
        for i in range(0, 100, 1):  # Using integers and dividing by 100
            i = i / 100.0
            buy_rate.append(i)
            returns_Buy.append(Trading_Strategy.calculate_total_profit(Trading_Strategy.trail_function(optimized_sell_rate, i)))
            num_transactions_Buy.append(len(Trading_Strategy.trail_function(optimized_sell_rate, i)))
    
        # Calculate transaction index for buy rates
        transaction_index_Buy = [
            (returns_Buy[j] / num_transactions_Buy[j]) if num_transactions_Buy[j] != 0 else 0 
            for j in range(len(buy_rate))
        ]
    
        # Find the optimized buy rate
        transaction_index_array_Buy = np.array(transaction_index_Buy)
        index = np.where(transaction_index_array_Buy == np.max(transaction_index_array_Buy))[0]
        optimized_buy_rate = buy_rate[index[0]]

        total_optimized_return = Trading_Strategy.calculate_total_profit(Trading_Strategy.trail_function(optimized_sell_rate, optimized_buy_rate))
        num_transaction = len(Trading_Strategy.trail_function(optimized_sell_rate, optimized_buy_rate))



        return {"optimized_sell_rate":optimized_sell_rate, 
                "optimized_buy_rate":optimized_buy_rate, 
                "returns":returns, 
                "returns_Buy":returns_Buy, 
                "num_transactions":num_transactions, 
                "num_transactions_Buy":num_transactions_Buy, 
                "transaction_index":transaction_index, 
                "transaction_index_Buy":transaction_index_Buy, 
                "total_optimized_return":total_optimized_return, 
                "num_transaction": num_transaction}
    
    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def index_visualization(self):
        plt.figure(figsize=(10, 3))  # Adjust size if necessary
        
        # Plot returns on the left y-axis
        plt.plot(trading_strategy.optimize_trading_rates()["returns"], label='returns')
        plt.xlabel('Time')
        plt.ylabel('Returns')
        
        # Create a twin y-axis to plot num_transactions on the right y-axis
        ax2 = plt.twinx()
        ax2.plot(trading_strategy.optimize_trading_rates()["num_transactions"], color='orange', label='num_transactions')
        ax2.set_ylabel('Number of Transactions')
        
        # Add title and legends
        plt.title('Time Series Comparison')
        plt.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        # Show plot
        plt.show()
        plt.figure(figsize=(10, 3))  # Adjust size if necessary
        
        # Plot returns on the left y-axis
        plt.plot(trading_strategy.optimize_trading_rates()["returns"], label='returns')
        plt.xlabel('Time')
        plt.ylabel('Returns')
        
        # Create a twin y-axis to plot num_transactions on the right y-axis
        ax2 = plt.twinx()
        ax2.plot(trading_strategy.optimize_trading_rates()["transaction_index"], color='orange', label='transaction_index')
        ax2.set_ylabel('transaction_index')
        
        # Add title and legends
        plt.title('Time Series Comparison')
        plt.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        # Show plot
        plt.show()
        
        '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''' 
        
        plt.figure(figsize=(10, 3))  # Adjust size if necessary
        
        # Plot returns on the left y-axis
        plt.plot(trading_strategy.optimize_trading_rates()["returns_Buy"], label='returns')
        plt.xlabel('Time')
        plt.ylabel('Returns')
        
        # Create a twin y-axis to plot num_transactions on the right y-axis
        ax2 = plt.twinx()
        ax2.plot(trading_strategy.optimize_trading_rates()["num_transactions_Buy"], color='orange', label='num_transactions')
        ax2.set_ylabel('Number of Transactions')
        
        # Add title and legends
        plt.title('Time Series Comparison')
        plt.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        # Show plot
        plt.show()
        
        plt.figure(figsize=(10, 3))  # Adjust size if necessary
        
        # Plot returns on the left y-axis
        plt.plot(trading_strategy.optimize_trading_rates()["returns_Buy"], label='returns')
        plt.xlabel('Time')
        plt.ylabel('Returns')
        
        # Create a twin y-axis to plot num_transactions on the right y-axis
        ax2 = plt.twinx()
        ax2.plot(trading_strategy.optimize_trading_rates()["transaction_index_Buy"], color='orange', label='transaction_index')
        ax2.set_ylabel('transaction_index')
        
        # Add title and legends
        plt.title('Time Series Comparison')
        plt.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        # Show plot
        plt.show()

    ####################### ####################### ####################### ####################### ####################### ####################### ########
    def transactions_visualization(self, MA_Window=5, transactions=None, title=None, picture_saving_path=None, batch_prices=None):
        data = self.data
        trading_strategy = TradingStrategy(data)
        
        if transactions == None:    
            transactions = trading_strategy.trail_function(trading_strategy.optimize_trading_rates()["optimized_sell_rate"], 
                                                           trading_strategy.optimize_trading_rates()["optimized_buy_rate"])
            
        # Calculate the moving average with a window of 5
        moving_average = data.rolling(window=MA_Window).mean()
        
        # Find the indices of the transaction prices in the time series
        buy_indices = []
        sell_indices = []
        buy_prices = []
        sell_prices = []
        
        for transaction in transactions:
            transaction_type, transaction_price = transaction
            # Find the closest index for each transaction price
            index = (np.abs(data - transaction_price)).idxmin() 
                                                                 
            if transaction_type.lower() == 'buy':
                buy_indices.append(index)
                buy_prices.append(data[index])
            elif transaction_type.lower() == 'sell':
                sell_indices.append(index)
                sell_prices.append(data[index])
        
        # Plot the time series
        plt.figure(figsize=(17, 10))
        plt.plot(data, label='Price', color='blue')
        
        # Plot the moving average
        plt.plot(moving_average, label='Moving Average (Window=5)', color='orange')
        
        # Add vertical lines for batch boundaries if batch_prices is provided
        if batch_prices is not None:
            for price in batch_prices:
                # Find indices where the price crosses this boundary
                # We use np.where to find all crossings
                indices = np.where(np.diff(np.signbit(data - price)))[0]
                
                for idx in indices:
                    plt.axvline(x=idx, color='gray', linestyle='--', alpha=0.5, linewidth=0.8, 
                              label='Batch Boundary' if idx == indices[0] else '')
        
        # Plot buy signals
        plt.scatter(buy_indices, buy_prices, marker='^', color='green', s=100, label='Buy Signal')
        plt.scatter(sell_indices, sell_prices, marker='v', color='red', s=100, label='Sell Signal')
        plt.xlabel('Time')
        plt.ylabel('Price')
        
        if title == None:
            plt.title('Time Series with Buy and Sell Signals')
        elif title != None: 
            plt.title(title)
            
        plt.legend()
        
        # Save the plot (optional, if you need to save it)
        if picture_saving_path != None:
            filename = os.path.join(picture_saving_path, f"{title}.png")
            plt.savefig(filename)
        
        # Show plot
        plt.show()

######################### ###################### ########################## ###################### ######################## #######################

    def moving_window_optimization(self, ma_ranges=(10,40)):
        data = self.data
        trading_strategy = TradingStrategy(data)
        # Starting Grownds
        transaction_index_ls = []
        moving_window_size = []
        
        for i in range(ma_ranges[0], ma_ranges[1]):
            # Determine the batch size
            moving_window = i
            transaction_index_batch_ls = []
            
            # Split the data into batches of moving_window entries
            batches = self.batch_from_end(lst=data,batch_size=i)
            if len(batches[0]) != i:
                batches = batches[1:]
                
            # Access each batch
            for batch in batches:
                trading_strategy = TradingStrategy(batch)
                returns = trading_strategy.calculate_total_profit(trading_strategy.trail_function(trading_strategy.optimize_trading_rates()["optimized_sell_rate"], 
                                                                                                  trading_strategy.optimize_trading_rates()["optimized_buy_rate"]))
                num_transactions = trading_strategy.optimize_trading_rates()["num_transaction"]
        
                if num_transactions != 0:
                    transaction_index_batch_ls.append(returns / num_transactions)
        
                elif num_transactions == 0: 
                    transaction_index_batch_ls.append(0)
            
            # Saving the moving_window result
            transaction_index_ls.append(np.mean(transaction_index_batch_ls))
            moving_window_size.append(moving_window)
            
        # Saving the optimized_moving_window
        transaction_index_ls_array = np.array(transaction_index_ls)
        index = np.where(transaction_index_ls_array == np.max(transaction_index_ls))[0]
        optimized_moving_window = moving_window_size[index[0]]
        optimized_moving_window

        # Creating the optimized batches
        batches = [data[j:j + optimized_moving_window] for j in range(0, len(data), optimized_moving_window)]
        batches = [batch.reset_index(drop=True) for batch in batches]

        # Creating the batch start and end lines by getting the prices at which it happens
        batch_prices = trading_strategy.batch_separator(list_of_batches=batches)
        
        total_trades = []
        for batch in batches:
            trading_strategy = TradingStrategy(batch)
            trades = trading_strategy.trail_function(trading_strategy.optimize_trading_rates()["optimized_sell_rate"], 
                                                     trading_strategy.optimize_trading_rates()["optimized_buy_rate"])

            total_trades = total_trades + trades
            
        total_batches_return = trading_strategy.calculate_total_profit(total_trades) 
        
        return {"total_trades":total_trades,
                "transaction_index_ls_array":transaction_index_ls_array,
                "optimized_moving_window":optimized_moving_window, 
                "moving_window_size":moving_window_size, 
                "total_batches_return":total_batches_return,
                "batch_prices": batch_prices, 
                "batches": batches
               }
    
    ############################# ################################## ############################### ############################### ####################### ##########################
    
    def kairos(self, stock_ticker, trades_1_0=None, graph_pic_saving_path=None):
        data = self.data
        trading_strategy = TradingStrategy(data)

        optimized_moving_window = trading_strategy.moving_window_optimization()["optimized_moving_window"]
        print(optimized_moving_window)
        
        plt.figure(figsize=(10, 5))
        plt.plot(trading_strategy.moving_window_optimization()["transaction_index_ls_array"], 
                 label='transaction_index_ls', marker='o')
        # plt.plot(trading_strategy.moving_window_optimization()["moving_window_size"], label='moving_window_size', marker='x')
        
        # Adding titles and labels
        plt.title('Line Plot of Two Columns')
        plt.xlabel('Index')
        plt.ylabel('Values')
        plt.legend()
        plt.grid()
        
        # Show plot
        plt.show()

        # Saving the rates for the last moving window, later to be used for the TITLE
        result = trading_strategy.optimize_trading_rates(data_external=data[-optimized_moving_window:].reset_index()[stock_ticker])
        returns = trading_strategy.calculate_total_profit(trading_strategy.moving_window_optimization()["total_trades"])       # trading_strategy.moving_window_optimization(["total_trades"]
        #returns = trading_strategy.moving_window_optimization()["total_batches_return"]
        
        title = f"{stock_ticker}_Buy Rate_{result['optimized_buy_rate']}__Sell Rate_{result['optimized_sell_rate']}__Win{optimized_moving_window}__PrfS_{round(result['total_optimized_return'], 2)}_PrfL_{round(returns, 2)}"  #PrfL_{returns}
        batch_prices = trading_strategy.moving_window_optimization()["batch_prices"]

    
        # Transaction Visualization
        trading_strategy.transactions_visualization(transactions=trading_strategy.moving_window_optimization()["total_trades"], 
                                                    title=title, 
                                                    picture_saving_path=graph_pic_saving_path, 
                                                    batch_prices=batch_prices)
        
        if trades_1_0 != None:
            trades = trading_strategy.moving_window_optimization()["total_trades"]
            print(trading_strategy.calculate_total_profit(trades))
            print(" ")
            print(trading_strategy.calculate_profit(trades))
            print(" ")
            print(batch_prices)
            #print(" ")
            #print(trading_strategy.moving_window_optimization()["batches"])

################################# ############################### ################################## ############################### #################

In [ ]:
data = pd.read_excel(r"C:\Users\DELL\Desktop\MA Model Data\Model Data All_SOX.xlsx")
#data = data["Close"]
data = data#[-300:-200].reset_index()
#data = data["ADI"]
stock_ticker = "NVDA"
stock_data = data[stock_ticker]

tradingStrategy = TradingStrategy(stock_data)
result = tradingStrategy.kairos(stock_ticker=stock_ticker, 
                                trades_1_0=1,
                                graph_pic_saving_path = r"C:\Users\DELL\Desktop\LSTM Data\MA Model\Pics3")


In [38]:
[50.0, 37.61574074, 28.93518519, 20.76367455, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


[0     50.000000
1     55.000000
2     60.000000
3     58.000000
4     57.000000
5     59.000000
6     60.000000
7     59.000000
8     62.000000
9     65.000000
10    70.000000
11    72.000000
12    70.000000
13    69.000000
14    65.000000
15    62.000000
16    55.000000
17    53.000000
18    41.666667
19    44.166667
20    45.833333
21    41.666667
22    45.833333
23    50.000000
24    48.333333
25    47.500000
26    49.166667
27    50.000000
28    49.166667
29    51.666667
30    54.166667
31    58.333333
32    60.000000
33    58.333333
34    57.500000
35    54.166667
36    51.666667
37    45.833333
38    36.805556
Name: NVDA2, dtype: float64, 0     34.722222
1     36.805556
2     38.194444
3     34.722222
4     38.194444
5     41.666667
6     40.277778
7     39.583333
8     40.972222
9     41.666667
10    40.972222
11    43.055556
12    45.138889
13    48.611111
14    50.000000
15    48.611111
16    47.916667
17    45.138889
18    43.055556
19    38.194444
20    36.805556
21    34.722222
22    30.671296
23    31.828704
24    28.935185
25    31.828704
26    34.722222
27    33.564815
28    32.986111
29    34.143519
30    34.722222
31    34.143519
32    35.879630
33    37.615741
34    40.509259
35    41.666667
36    40.509259
37    39.930556
38    37.615741
Name: NVDA2, dtype: float64, 0     35.879630
1     31.828704
2     25.559414
3     24.112654
4     25.559414
5     26.523920
6     24.112654
7     26.523920
8     28.935185
9     27.970679
10    27.488426
11    28.452932
12    28.935185
13    28.452932
14    29.899691
15    31.346451
16    33.757716
17    34.722222
18    33.757716
19    33.275463
20    31.346451
21    29.899691
22    26.523920
23    21.299511
24    20.093879
25    21.299511
26    22.103266
27    20.093879
28    22.103266
29    24.112654
30    23.308899
31    22.907022
32    23.710777
33    24.112654
34    23.710777
35    24.916409
36    26.122042
37    28.131430
38    28.935185
Name: NVDA2, dtype: float64, 0     28.131430
1     27.729552
2     26.122042
3     24.916409
4     22.103266
5     21.299511
6     20.093879
7     21.299511
8     22.103266
9     20.093879
10    22.103266
11    24.112654
12    23.308899
13    22.907022
14    23.710777
15    24.112654
16    23.710777
17    24.916409
18    26.122042
19    28.131430
20    28.935185
21    28.131430
22    27.729552
23    26.122042
24    24.916409
25    22.103266
26    21.299511
27    20.093879
28    17.749593
29    18.419389
30    16.744899
31    18.419389
32    20.093879
33    19.424083
34    19.089185
35    19.758981
36    20.093879
37    19.758981
38    20.763675
Name: NVDA2, dtype: float64, 0     21.768368
1     23.442858
2A2, dtype: float64, 0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
Name: NVDA2, dtype: float64, 0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: NVDA2, dtype: float64]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (109912245.py, line 4)

In [41]:
[56.66666667, 50.71759259, 34.91512346, 21.16555213, 27.52861368, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
 
[0     56.666667
1     58.333333
2     59.000000
3     59.333333
4     60.333333
5     62.500000
6     64.666667
7     66.333333
8     68.000000
9     68.500000
10    68.000000
11    65.500000
12    62.333333
13    57.611111
14    53.472222
15    50.277778
16    46.888889
17    45.361111
18    44.861111
19    45.972222
20    46.527778
21    47.083333
22    48.472222
23    49.027778
24    49.305556
25    50.277778
26    52.083333
27    53.888889
28    55.277778
29    56.666667
30    57.083333
31    56.666667
32    54.583333
33    50.717593
Name: NVDA4, dtype: float64, 0     46.782407
1     43.333333
2     40.671296
3     37.847222
4     36.574074
5     37.384259
6     38.310185
7     38.773148
8     39.236111
9     40.393519
10    40.856481
11    41.087963
12    41.898148
13    43.402778
14    44.907407
15    46.064815
16    47.222222
17    47.569444
18    47.222222
19    45.486111
20    43.287037
21    40.972222
22    38.097994
23    35.879630
24    33.526235
25    32.465278
26    32.118056
27    31.925154
28    32.310957
29    32.696759
30    33.661265
31    34.047068
32    34.239969
33    34.915123
Name: NVDA4, dtype: float64, 0     36.168981
1     37.422840
2     38.387346
3     39.351852
4     39.641204
5     39.351852
6     37.905093
7     35.220550
8     32.487783
9     30.092593
10    28.243956
11    26.282793
12    25.398663
13    25.961291
14    26.604295
15    26.925797
16    27.247299
17    28.051055
18    28.372557
19    28.533308
20    29.095936
21    30.140818
22    31.185700
23    31.989455
24    32.793210
25    33.034336
26    32.793210
27    31.587577
28    29.350459
29    27.073152
30    25.077161
31    23.536630
32    21.902328
33    21.165552
Name: NVDA4, dtype: float64, 0     21.634409
1     22.170246
2     22.438164
3     22.706083
4     23.375879
5     23.643797
6     23.777756
7     24.246614
8     25.117348
9     25.988083
10    26.657879
11    27.327675
12    27.528614
13    27.327675
14    26.322981
15    25.050369
16    23.710777
17    22.639103
18    21.969307
19    21.165552
20    21.165552
21    21.634409
22    22.170246
23    22.438164
24    22.706083
25    23.375879
26    23.643797
27    23.777756
28    24.246614
29    25.117348
30    25.988083
31    26.657879
32    27.327675
33    27.528614
Name: NVDA4, dtype: float64, 0     27.327675
1     26.322981
2     25.050369
3     23.710777
4     22.047450
5     20.763675
6     19.401756
7     18.787776
8     18.586838
9     18.475205
10    18.698470
11    18.921736
12    19.479899
13    19.703164
14    19.814797
15    20.205511
16    20.931124
17    21.656736
18    22.214899
19    22.773062
20    22.940511
21    22.773062
22    21.935817
23    20.875307
24    19.758981
25    18.865919
26    18.307756

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1153306617.py, line 3)